In [1]:
import numpy as np
import cv2
from Detector_COV.detection_class import detector_class

from scripts.preprocessor_functions import remove_vignette

import os
import json
from skimage.morphology import disk
import time
import json


In [2]:
SET_NAME = "Set_10"
MATERIAL = "Graphene"
SET_DIRS = r"C:\Users\Uslu.INSTITUT2B\Desktop\Annotated_Images"

SET_DIR = os.path.join(SET_DIRS, MATERIAL,SET_NAME)
IMAGE_DIR = os.path.join(SET_DIR, "Images")
FLATFIELD = os.path.join(SET_DIR, "flatfield.png")
PARAMETER_PATH = os.path.join(SET_DIR, "gaussian_mixture_model_contrast_data.json")
DETECTED_SAVE_DIR = os.path.join(SET_DIR, "Detected_Images")

In [4]:
with open(PARAMETER_PATH) as f:
    contrast_params = json.load(f)

SIZE_THRESHOLD = 100
MAGNIFICATION = 20
STANDARD_DEVIATION_THRESHOLD = 5
COVARIANCE_SCALING_FACTORS = [1, 1, 1]
USED_CHANNELS = [0, 1, 2]


myDetector = detector_class(
    contrast_dict=contrast_params,
    size_threshold=SIZE_THRESHOLD,
    magnification=MAGNIFICATION,
    standard_deviation_threshold=STANDARD_DEVIATION_THRESHOLD,
    used_channels=USED_CHANNELS,
    covariance_scaling_factors=COVARIANCE_SCALING_FACTORS,
)

In [5]:
FF = cv2.imread(FLATFIELD)

IMAGE_NAMES = os.listdir(IMAGE_DIR)

current_image_number = 0
current_flake_number = 0
for image_name in IMAGE_NAMES:
    image = cv2.imread(os.path.join(IMAGE_DIR, image_name))

    image = remove_vignette(image, FF)

    start = time.time()
    detected_flakes = myDetector.detect_flakes(image)
    print(
        f"Time : {np.round(time.time() - start,3)}s | Detected flakes : {len(detected_flakes)} Name : {image_name}"
    )
    # Operation on the flakes
    if len(detected_flakes) != 0:
        current_image_number += 1
        all_flake_mask = np.zeros(image.shape[:2], dtype=np.uint8)

        # Operate on each Flake
        for flake in detected_flakes:
            current_flake_number += 1

            flake_mask = flake["mask"]
            flake_thickness = flake["layer"]

            flake_thickness_dir = os.path.join(SET_DIR, f"Masks_{flake_thickness}")
            if not os.path.exists(flake_thickness_dir):
                os.makedirs(flake_thickness_dir)

            flake_mask_path = os.path.join(
                flake_thickness_dir, f"MASK_{current_flake_number}_{image_name}"
            )

            cv2.imwrite(flake_mask_path, flake_mask)

            all_flake_mask = cv2.bitwise_or(all_flake_mask, flake_mask, all_flake_mask)

            # # Extract some data from the Dict
            (x, y) = flake["position_bbox"]
            w = flake["width_bbox"]
            h = flake["height_bbox"]
            # cv2.rectangle(
            #     image,
            #     (x - 20, y - 20),
            #     (x + w + 20, y + h + 20),
            #     color=[0, 0, 0],
            #     thickness=2,
            # )

            # Dilate the flake outline and get the gradient to get a nice border
            # outline_flake = cv2.dilate(flake["mask"], disk(2))
            outline_flake = cv2.morphologyEx(
                flake_mask,
                cv2.MORPH_GRADIENT,
                disk(1),
            )

            # Draw this border on the image
            image[outline_flake != 0] = [0, 0, 255]

            cv2.putText(
                image,
                f"{flake_thickness}",
                (x + w // 2, y + h // 2),
                cv2.FONT_HERSHEY_SIMPLEX,
                thickness=2,
                fontScale=2,
                color=(0, 0, 0),
            )
            #####

        # Now save the Image to its new Home
        cv2.imwrite(
            os.path.join(DETECTED_SAVE_DIR, f"Detected_{image_name}"),
            image,
        )


Time : 1.043s | Detected flakes : 0 Name : 00c92be5-4edb-485a-9f7e-6d7b4c7d2abc.png
Time : 0.308s | Detected flakes : 0 Name : 03b240ea-ea52-4978-a742-bb2aa45e8537.png
Time : 1.611s | Detected flakes : 25 Name : 043e55f8-7250-48d7-a8be-a4d39567cb39.png
Time : 0.718s | Detected flakes : 7 Name : 04ce8e66-7212-4774-bbfa-9829d22447c7.png
Time : 0.483s | Detected flakes : 4 Name : 04cf8489-b611-4cd4-8d40-ae0b26a1fe8b.png
Time : 0.336s | Detected flakes : 3 Name : 0a0229c0-427d-4552-877c-194b97c661fb.png
Time : 0.626s | Detected flakes : 6 Name : 0afb026f-5232-4b35-ab5c-de90ed083724.png
Time : 0.544s | Detected flakes : 6 Name : 0c9c3d0a-cd06-429a-989b-1b8fcd5eb124.png
Time : 0.277s | Detected flakes : 1 Name : 0d1519b0-4085-4cf7-a42d-b270694c7171.png
Time : 0.579s | Detected flakes : 3 Name : 0d875d40-820d-4f4d-8aac-dc3415a830cb.png
Time : 0.393s | Detected flakes : 1 Name : 10acf238-cf1d-4175-917d-ef0405484794.png
Time : 0.42s | Detected flakes : 1 Name : 112af009-07cd-4124-b3d1-993dbadec